In [ ]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite' #same as integer8  hardware deployable model...also called quantised  hardware deployable model
MODEL_TFLITE = MODELS_DIR + "model.tflite" #same as floating point hardware deployable model ...also called non-quantised  hardware deployable model


MODEL_TFLITE_MICRO = MODELS_DIR + 'model_tflite.cc'

In [ ]:

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
# Set seed for experiment reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
tf.__version__

In [ ]:
#let's download the dataset from UCI
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'

#columns don't come named, so these are the names for use
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

#turn it into a Pandas Dataset
dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
#drop nulls
dataset = dataset.dropna()

#train / test split
train_dataset = dataset.sample(frac=0.8, random_state=seed)
test_dataset = dataset.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()


#MPG is our target
train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

#select only the Horsepower column
#train_features = np.array(train_features['Horsepower'])
#test_features = np.array(test_features['Horsepower'])
reject1 = train_features.pop('Model Year')
reject2 = train_features.pop('Origin')
reject1 = test_features.pop('Model Year')
reject2 = test_features.pop('Origin')


train_features = np.array(train_features)
test_features = np.array(test_features)
test_features_ = np.zeros(test_features.shape)
train_features_ = np.zeros(train_features.shape)
train_labels_ = np.zeros(train_labels.shape)
test_labels_ = np.zeros(test_labels.shape)

#Feature Scaling
for i in range(5):
  max = np.max(train_features[:,i]);
  min = np.min(train_features[:,i]);
  train_features_[:,i] = (train_features[:,i] - min)/(max-min);
train_features = train_features_;


for i in range(5):
  max = np.max(test_features[:,i]);
  min = np.min(test_features[:,i]);
  test_features_[:,i] = (test_features[:,i] - min)/(max-min);
test_features = test_features_;


max = np.max(train_labels);
min = np.min(train_labels);
train_labels_ = (train_labels - min)/(max-min);
max = np.max(test_labels);
min = np.min(test_labels);
test_labels_ = (test_labels - min)/(max-min);
train_labels = train_labels_;
test_labels = test_labels_;


print(test_features_[13])
print(test_labels_[13])
print(train_features[34])
print(train_labels[34])

plt.plot(train_features, train_labels, 'b.', label='Actual values')

In [ ]:
model = tf.keras.Sequential()

#only takes one feature
model.add(keras.layers.Dense(16, activation='relu', input_shape=(5,)))
model.add(keras.layers.Dense(16, activation='relu'))
#only one output
model.add(keras.layers.Dense(1))

model.compile(optimizer='adam', loss="mse", metrics=["mae"])
# Train the model
history = model.fit(train_features, train_labels, epochs=1000, batch_size=64,
                    validation_data=(test_features, test_labels))



# Save the model to disk
model.save(MODEL_TF)

In [ ]:
index=4;
print(test_features[4])
model.predict(test_features[[4]])

In [ ]:
#plotting loss over our epochs
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(10, 4))


plt.plot(epochs, train_loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
# Calculate and print the loss on our test dataset
test_loss, test_mae = model.evaluate(test_features, test_labels)

# Make predictions based on our test dataset
y_test_pred = model.predict(test_features)

# Graph the predictions against the actual values
plt.clf()
plt.title('Comparison of predictions and actual values')
plt.plot(test_features, test_labels, 'b.', label='Actual values')
plt.plot(test_features, y_test_pred, 'r.', label='TF predicted')
plt.legend()
plt.show()